In [54]:
import pandas as pd
import nltk
# nltk.download('omw-1.4')
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
from nltk.parse.stanford import StanfordDependencyParser

In [56]:
!brew install wget

Updating Homebrew...
==> Auto-updated Homebrew!
Updated 3 taps (homebrew/cask-versions, homebrew/core and homebrew/cask).
==> New Formulae
abi-compliance-checker                   libgrape-lite
abi-dumper                               libsoup@2
arkade                                   lighthouse
bash-language-server                     lua-language-server
biber                                    mcfly
brigade-cli                              mist
btop                                     mold
cava                                     mt32emu
chroma                                   nerdctl
coursier                                 netmask
cpptoml                                  openliberty-jakartaee9
ctlptl                                   openliberty-webprofile9
cwb3                                     pam-reattach
djhtml                                   payload-dumper-go
dtm                                      php@8.0
dynomite                                 pinot
elixir-ls         

In [55]:
!wget https://nlp.stanford.edu/software/stanford-corenlp-4.2.2.zip
!wget https://nlp.stanford.edu/software/stanford-corenlp-4.2.2-models-english.jar
!unzip /content/stanford-corenlp-4.2.2.zip

<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 3.2 Final//EN">
<title>Redirecting...</title>
<h1>Redirecting...</h1>
<p>You should be redirected automatically to target URL: <a href="https://downloads.cs.stanford.edu/nlp/software/stanford-corenlp-4.2.2.zip">https://downloads.cs.stanford.edu/nlp/software/stanford-corenlp-4.2.2.zip</a>.  If not click the link.<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 3.2 Final//EN">
<title>Redirecting...</title>
<h1>Redirecting...</h1>
<p>You should be redirected automatically to target URL: <a href="https://downloads.cs.stanford.edu/nlp/software/stanford-corenlp-4.2.2-models-english.jar">https://downloads.cs.stanford.edu/nlp/software/stanford-corenlp-4.2.2-models-english.jar</a>.  If not click the link.unzip:  cannot find or open /content/stanford-corenlp-4.2.2.zip, /content/stanford-corenlp-4.2.2.zip.zip or /content/stanford-corenlp-4.2.2.zip.ZIP.


In [47]:
path_train = "SEM-2012-SharedTask-CD-SCO-training-simple.v2.txt"
data = pd.read_csv(path_train, sep="\t", header=None, names=['annotator', 'sentence_id', 'token_id', 'token', 'label'])
data

,annotator,sentence_id,token_id,token,label
0,baskervilles01,0,0,Chapter,O
1,baskervilles01,0,1,1.,O
2,baskervilles01,0,2,Mr.,O
3,baskervilles01,0,3,Sherlock,O
4,baskervilles01,0,4,Holmes,O
...,...,...,...,...,...
65446,baskervilles14,270,58,slopes,O
65447,baskervilles14,270,59,of,O
65448,baskervilles14,270,60,the,O
65449,baskervilles14,270,61,moor,O


## Extract features

### Lemma, POS tag, HasApostrophe, HasNegationPrefix, HasNegationSuffix, HasNegationInfix, IsPartOfNegation

### Bug: hasApostrophe selects as TRUE intances that have a quote character <">
### Sol: Clean text from quotes or force lambda to detect just one <'>

### Bug: pos tags of token <.> is <.>. Should we remove them?

### https://stackoverflow.com/questions/62439727/how-can-i-detect-if-it-is-a-negation-with-a-verb-in-a-sentence-using-spacyor-ot

In [48]:


lemmatizer = WordNetLemmatizer()

column_values = data[['annotator']].values.ravel()
annotator_ids = pd.unique(column_values)
print('Ids of annotators :', annotator_ids)

# data['token'] = data['token'].apply(lambda x: x.lower()) # lowercase each token
data['lemma'] = data['token'].apply(lambda x: lemmatizer.lemmatize(x)) # lemmatize each token
# data['lemma'] = data['lemma'].apply(lambda x: x.lower()) # lowercase each lemma
data['hasApostrophe'] = data['token'].apply(lambda x: '\'' in x) # check if token has apostrophe


column_values = data[['annotator']].values.ravel()
annotator_names = pd.unique(column_values)

dependency_parser = StanfordDependencyParser(path_to_jar=path_to_jar, path_to_models_jar=path_to_models_jar)


pos_tags = []
for annotator in annotator_names:
    annotator_data = data[data['annotator'] == annotator]
    column_values = annotator_data[['sentence_id']].values.ravel()
    sentence_ids = pd.unique(column_values)
    for sent_id in sentence_ids:
        sentence = annotator_data.loc[annotator_data['sentence_id'] == sent_id, 'token']
        for elem in nltk.pos_tag(sentence):
            pos_tags.append(elem[1])

data['pos-tag'] = pos_tags
data

# Error example
# data.loc[ data['sentence_id'] == 12]
# data.loc[(data['annotator'] == 'baskervilles01') & (data['sentence_id'] == 12)]



Ids of annotators : ['baskervilles01' 'baskervilles02' 'baskervilles03' 'baskervilles04'
 'baskervilles05' 'baskervilles06' 'baskervilles07' 'baskervilles08'
 'baskervilles09' 'baskervilles10' 'baskervilles11' 'baskervilles12'
 'baskervilles13' 'baskervilles14']


,annotator,sentence_id,token_id,token,label,lemma,hasApostrophe,pos-tag
0,baskervilles01,0,0,Chapter,O,Chapter,False,NN
1,baskervilles01,0,1,1.,O,1.,False,CD
2,baskervilles01,0,2,Mr.,O,Mr.,False,NNP
3,baskervilles01,0,3,Sherlock,O,Sherlock,False,NNP
4,baskervilles01,0,4,Holmes,O,Holmes,False,NNP
...,...,...,...,...,...,...,...,...
65446,baskervilles14,270,58,slopes,O,slope,False,NNS
65447,baskervilles14,270,59,of,O,of,False,IN
65448,baskervilles14,270,60,the,O,the,False,DT
65449,baskervilles14,270,61,moor,O,moor,False,NN


In [3]:
from nltk.parse.stanford import StanfordDependencyParser

path_to_jar = 'path_to/stanford-parser-full-2014-08-27/stanford-parser.jar'
path_to_models_jar = 'path_to/stanford-parser-full-2014-08-27/stanford-parser-3.4.1-models.jar'

dependency_parser = StanfordDependencyParser()

result = dependency_parser.raw_parse('I shot an elephant in my sleep')
dep = result.next()

list(dep.triples())

/var/folders/v9/cd3cfwkx541gs96n95styvwr0000gn/T/ipykernel_9536/501562092.py:6: DeprecationWarning: The StanfordDependencyParser will be deprecated
Please use nltk.parse.corenlp.CoreNLPDependencyParser instead.
  dependency_parser = StanfordDependencyParser()


LookupError: 

===========================================================================
  NLTK was unable to find stanford-parser\.jar! Set the CLASSPATH
  environment variable.

  For more information, on stanford-parser\.jar, see:
    <https://nlp.stanford.edu/software/lex-parser.shtml>
===========================================================================